In [1]:
# Dependencies
from TwitterAPI import TwitterAPI
import json
import pandas as pd

In [2]:
# Login using credentials stored in auth.json file

# Define authentication file path
auth_path = 'data/auth.json'

# Read auth.json file
with open(auth_path, 'r') as file:
    auth_dict = json.load(file)

# Execute login
api = TwitterAPI(**auth_dict)

# Verify credentials
req = api.request('account/verify_credentials')
assert req.status_code == 200, 'Authentication failed!'

In [3]:
# Retrieve tweets for @FeminismInIndia
res = api.request('search/tweets', {
    'q': 'from:FeminismInIndia',
    'tweet_mode': 'extended'
})

# Print response status
print('Response status: {:d}'.format(res.status_code))
print()

# Store retireved tweets
tweets = [tweet for tweet in res]
tweets

Response status: 200



[{'created_at': 'Tue Dec 24 21:18:47 +0000 2019',
  'id': 1209584228578529285,
  'id_str': '1209584228578529285',
  'full_text': 'Why Does BJP Government Fear Akhil\xa0Gogoi? https://t.co/GSZ45Lc6k4 https://t.co/bciZdbjLlg',
  'truncated': False,
  'display_text_range': [0, 65],
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [],
   'urls': [{'url': 'https://t.co/GSZ45Lc6k4',
     'expanded_url': 'https://feminisminindia.com/2019/12/25/wbjp-government-fear-akhil-gogoi/',
     'display_url': 'feminisminindia.com/2019/12/25/wbj…',
     'indices': [42, 65]}],
   'media': [{'id': 1209584227051851783,
     'id_str': '1209584227051851783',
     'indices': [66, 89],
     'media_url': 'http://pbs.twimg.com/media/EMlOciNXUAcdOJq.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/EMlOciNXUAcdOJq.jpg',
     'url': 'https://t.co/bciZdbjLlg',
     'display_url': 'pic.twitter.com/bciZdbjLlg',
     'expanded_url': 'https://twitter.com/FeminismInIndia/status/12095842285785

In [4]:
# Define attributes which will be kept from retrieved tweets
kept_attr = ('created_at', 'id_str', 'full_text', 'text', 'truncated', 'entities', 'geo', 'coordinates',
             'place', 'retweet_count', 'favourite_count', 'in_reply_to_status_id_str', 
             'in_reply_to_user_id_str', 'lang')

# Apply filter
for i, tweet in enumerate(tweets):
    # Handle entities
    hashtags = []
    for hashtag in tweet['entities']['hashtags']:
        hashtags.append(hashtag['text'])
    tweet['entities'] = hashtags
    # Handle text
    tweet['text'] = tweet['full_text' if 'full_text' in set(tweet.keys()) else 'text']
    # Substitute i-th tweet
    tweets[i] = {k: tweet.get(k, None) for k in kept_attr}
    # Show i-th tweet
    print(tweets[i])
    print()

{'created_at': 'Tue Dec 24 21:18:47 +0000 2019', 'id_str': '1209584228578529285', 'full_text': 'Why Does BJP Government Fear Akhil\xa0Gogoi? https://t.co/GSZ45Lc6k4 https://t.co/bciZdbjLlg', 'text': 'Why Does BJP Government Fear Akhil\xa0Gogoi? https://t.co/GSZ45Lc6k4 https://t.co/bciZdbjLlg', 'truncated': False, 'entities': [], 'geo': None, 'coordinates': None, 'place': None, 'retweet_count': 2, 'favourite_count': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id_str': None, 'lang': 'en'}

{'created_at': 'Tue Dec 24 18:45:10 +0000 2019', 'id_str': '1209545568990769152', 'full_text': 'What Is Section 144 And Why We Should Still Resist Against\xa0CAA https://t.co/De9RnuwjSM https://t.co/Dw3P5qT6rA', 'text': 'What Is Section 144 And Why We Should Still Resist Against\xa0CAA https://t.co/De9RnuwjSM https://t.co/Dw3P5qT6rA', 'truncated': False, 'entities': [], 'geo': None, 'coordinates': None, 'place': None, 'retweet_count': 0, 'favourite_count': None, 'in_reply_to_status_id_st

In [5]:
# Store tweets into Pandas DataFrame object
tweets = pd.DataFrame(tweets)
tweets.head()

,created_at,id_str,full_text,text,truncated,entities,geo,coordinates,place,retweet_count,favourite_count,in_reply_to_status_id_str,in_reply_to_user_id_str,lang
0,Tue Dec 24 21:18:47 +0000 2019,1209584228578529285,Why Does BJP Government Fear Akhil Gogoi? http...,Why Does BJP Government Fear Akhil Gogoi? http...,False,[],None,None,None,2,None,None,None,en
1,Tue Dec 24 18:45:10 +0000 2019,1209545568990769152,What Is Section 144 And Why We Should Still Re...,What Is Section 144 And Why We Should Still Re...,False,[],None,None,None,0,None,None,None,en
2,Tue Dec 24 18:40:56 +0000 2019,1209544503649955840,14 Queer Moments That Made 2019 Memorable http...,14 Queer Moments That Made 2019 Memorable http...,False,[],None,None,None,2,None,None,None,en
3,Tue Dec 24 18:35:28 +0000 2019,1209543127738003460,Interview With Bindumadhav Khire On Media’s Co...,Interview With Bindumadhav Khire On Media’s Co...,False,[GBVInMedia],None,None,None,0,None,None,None,en
4,Tue Dec 24 17:30:00 +0000 2019,1209526652159565825,When we think of the minority groups who may b...,When we think of the minority groups who may b...,False,"[NRC, CAA, Adivasis]",None,None,None,14,None,None,None,en


In [6]:
# Save tweets to file
tweets.to_csv('data/database/tweets.csv')